In [1]:
import json
import textwrap
import random
import pandas as pd
import random
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings

from judges.graders.response_quality import MTBenchChatBotResponseQuality
from judges.graders.relevance import ReliableCIRelevance
from judges.classifiers.correctness import PollMultihopCorrectness

In [2]:
PROMPT = "provide a one paragraph summary of artificial intelligence"

REPSPONSE_TEXT = textwrap.dedent(
    """
    Artificial Intelligence (AI) is a branch of computer science focused on creating machines that can perform tasks requiring human-like intelligence, \nsuch as learning, reasoning, problem-solving, perception, and language understanding. It encompasses a wide range of techniques, including machine \nlearning, neural networks, and deep learning, which enable systems to recognize patterns, make decisions, and improve over time. AI is applied across \nvarious fields, from healthcare and finance to robotics and entertainment, driving innovation and efficiency. While AI presents significant \nopportunities, it also raises ethical concerns related to bias, privacy, and job displacement, necessitating careful regulation and responsible \ndevelopment.
    """
)

REFERENCE_TEXT = textwrap.dedent(
    """
    Artificial Intelligence (AI) refers to the development of computer systems that can perform tasks typically requiring human intelligence, such as \nlearning, reasoning, problem-solving, perception, and natural language understanding. AI is powered by techniques like machine learning, deep \nlearning, and neural networks, enabling it to analyze vast amounts of data, recognize patterns, and make autonomous decisions. While AI has \ntransformative applications in fields like healthcare, finance, and automation, it also raises major considerations. Ethical concerns include bias in \nalgorithms, data privacy, and the potential for job displacement. Additionally, ensuring AI systems are transparent, explainable, and aligned with \nhuman values is crucial for responsible development and deployment.
    """
)

## Helper Functions

In [3]:
# function to read in a paragraph and scrambe the words in each sentence
def scramble_paragraph(paragraph):
    sentences = paragraph.split(".")
    scrambled_sentences = []
    for sentence in sentences:
        words = sentence.split(" ")
        random.shuffle(words)
        scrambled_sentence = " ".join(words)
        scrambled_sentences.append(scrambled_sentence)
    scrambled_paragraph = ". ".join(scrambled_sentences)
    return scrambled_paragraph


# function to print a paragraph with out line breaks
# into lines of about 80 characters make sure to break at a white space
def print_paragraph(paragraph):
    lines = textwrap.wrap(paragraph, width=100)
    for line in lines:
        print(line)

# function that takes in a evaluator objecct and paragraph and
# runs the evaluator on the paragraph and prints the results
# then runs the evaluaton on the scambled paragraph and prints the results
def evaluate_paragraph(evaluator, prompt, paragraph, reference=None):
    print(f"prompt: {prompt}")
    print("Evaluating paragraph:")
    print_paragraph(paragraph)
    print()

    if reference:
        print("Evaluating with reference:")
        print_paragraph(reference)
        print()
        results = evaluator.judge(prompt, paragraph, expected=reference)
    else:
        print("Evaluating without reference")
        print()
        results = evaluator.judge(prompt, paragraph)
        
    print(f"SCORE: {results.score}")
    print(f"FEEDBACK:")
    print_paragraph(results.reasoning)
    


In [4]:
print(REPSPONSE_TEXT)


    Artificial Intelligence (AI) is a branch of computer science focused on creating machines that can perform tasks requiring human-like intelligence, 
such as learning, reasoning, problem-solving, perception, and language understanding. It encompasses a wide range of techniques, including machine 
learning, neural networks, and deep learning, which enable systems to recognize patterns, make decisions, and improve over time. AI is applied across 
various fields, from healthcare and finance to robotics and entertainment, driving innovation and efficiency. While AI presents significant 
opportunities, it also raises ethical concerns related to bias, privacy, and job displacement, necessitating careful regulation and responsible 
development.



In [5]:
print(scramble_paragraph(REPSPONSE_TEXT))

Intelligence 
 Artificial  science learning, computer can problem-solving, of machines on as that tasks perception, human-like branch focused  a and  reasoning, intelligence, understanding 
such requiring creating (AI) perform language is. to 
learning, enable time patterns, neural and over learning, networks, including encompasses range decisions, which a improve  systems machine and of make recognize It wide techniques, deep. across fields, entertainment, AI applied innovation and driving to and finance robotics from 
various is and efficiency  healthcare. job concerns ethical also While and responsible raises AI necessitating bias, it related to displacement, significant presents 
opportunities, privacy, regulation 
development and careful . 



## Response Quality

In [6]:
judge = MTBenchChatBotResponseQuality(model="gpt-4o")
evaluate_paragraph(judge, PROMPT, REPSPONSE_TEXT)
print("\n--------------------------------------------------\n")
evaluate_paragraph(judge, PROMPT, scramble_paragraph(REPSPONSE_TEXT))

prompt: provide a one paragraph summary of artificial intelligence
Evaluating paragraph:
     Artificial Intelligence (AI) is a branch of computer science focused on creating machines that
can perform tasks requiring human-like intelligence,  such as learning, reasoning, problem-solving,
perception, and language understanding. It encompasses a wide range of techniques, including machine
learning, neural networks, and deep learning, which enable systems to recognize patterns, make
decisions, and improve over time. AI is applied across  various fields, from healthcare and finance
to robotics and entertainment, driving innovation and efficiency. While AI presents significant
opportunities, it also raises ethical concerns related to bias, privacy, and job displacement,
necessitating careful regulation and responsible  development.

Evaluating without reference

SCORE: 9
FEEDBACK:
The AI assistant's response provides a concise and informative summary of artificial intelligence.
It effective

## RESPONSE CORRECTNESS

In [7]:
judge = PollMultihopCorrectness(model="gpt-4o")
evaluate_paragraph(judge, PROMPT, REPSPONSE_TEXT, reference=REFERENCE_TEXT)
print("\n--------------------------------------------------\n")
evaluate_paragraph(judge, PROMPT, scramble_paragraph(REPSPONSE_TEXT), reference=REFERENCE_TEXT)
print("\n--------------------------------------------------\n")
evaluate_paragraph(judge, PROMPT, scramble_paragraph(REPSPONSE_TEXT))

prompt: provide a one paragraph summary of artificial intelligence
Evaluating paragraph:
     Artificial Intelligence (AI) is a branch of computer science focused on creating machines that
can perform tasks requiring human-like intelligence,  such as learning, reasoning, problem-solving,
perception, and language understanding. It encompasses a wide range of techniques, including machine
learning, neural networks, and deep learning, which enable systems to recognize patterns, make
decisions, and improve over time. AI is applied across  various fields, from healthcare and finance
to robotics and entertainment, driving innovation and efficiency. While AI presents significant
opportunities, it also raises ethical concerns related to bias, privacy, and job displacement,
necessitating careful regulation and responsible  development.

Evaluating with reference:
     Artificial Intelligence (AI) refers to the development of computer systems that can perform
tasks typically requiring human inte

## Revelance

In [8]:
judge = ReliableCIRelevance(model="gpt-4o")
evaluate_paragraph(judge, PROMPT, REPSPONSE_TEXT)



prompt: provide a one paragraph summary of artificial intelligence
Evaluating paragraph:
     Artificial Intelligence (AI) is a branch of computer science focused on creating machines that
can perform tasks requiring human-like intelligence,  such as learning, reasoning, problem-solving,
perception, and language understanding. It encompasses a wide range of techniques, including machine
learning, neural networks, and deep learning, which enable systems to recognize patterns, make
decisions, and improve over time. AI is applied across  various fields, from healthcare and finance
to robotics and entertainment, driving innovation and efficiency. While AI presents significant
opportunities, it also raises ethical concerns related to bias, privacy, and job displacement,
necessitating careful regulation and responsible  development.

Evaluating without reference



NameError: name 'context' is not defined